# Behavior Cloning

### Data Augmentation and Formatting
Flip images and measurements

In [54]:
import csv
import cv2
import numpy as np

In [56]:
lines = []
with open("./data/driving_log.csv") as f:
    reader = csv.reader(f)
    for line in reader:
        filename = line[0].split('/')
        try:
            filename = '/'.join(['data',filename[-2],filename[-1]])
        except:
            print(filename, "::", line)
        lines.append([filename, *line[-4:]])

fw = open("./data/driving_log_updated.csv", "w")
for line in lines:
    fw.write("{},{},{},{},{}\n".format(*line))
    if abs(float(line[1])) > 0.3:
        new_filename = line[0][:-4]+"_flipped.jpg"
        img = cv2.imread(line[0], cv2.IMREAD_COLOR)
        img = np.fliplr(img)
        fw.write("{},{},{},{},{}\n".format(new_filename, -1.0 * float(line[1]), line[2], line[3], line[4]))
        cv2.imwrite(new_filename, img)
    
fw.close()

Crop Images

In [3]:
# 55 off top
# 30 off bottom
#fw = open("./data/driving_log_updated_cropped.csv", "w")
#with open("./data/driving_log_updated.csv") as f:
#    reader = csv.reader(f)
#    for line in reader:
#        filename = line[0]
#        img = cv2.imread(filename)
#        img = img[55:-30, :]
#        f_split = filename.split("/")
#        f_split[-2] = "CROPPED_IMG"
#        new_filename = '/'.join(f_split)
#        fw.write("{},{},{},{},{}\n".format(new_filename, *line[1:]))
#        cv2.imwrite(new_filename, img)
#    
#fw.close()

### Define Datasets

In [57]:
import random
import csv

TEST_SPLIT_RATIO = 0.2
VALIDATION_SPLIT_RATIO = 0.1

train_data = []
test_data = []
validation_data = []

with open("./data/driving_log_updated.csv") as f:
    reader = csv.reader(f)
    for line in reader:
        rand = random.random()
        if rand < TEST_SPLIT_RATIO:
            test_data.append(line)
        elif rand < VALIDATION_SPLIT_RATIO+TEST_SPLIT_RATIO:
            validation_data.append(line)
        else:
            train_data.append(line)


In [58]:
total = 1.0* len(train_data)+len(test_data)+len(validation_data)
print(len(train_data), len(train_data)/total )
print(len(test_data), len(test_data)/total)
print(len(validation_data), len(validation_data)/total)


11793 0.7041437783615954
3341 0.19948650585144495
1614 0.09636971578695963


In [60]:
import sklearn
import random

def generator(samples, batch_size=32):
    num_samples = len(samples)
    samples = sklearn.utils.shuffle(samples)
    offset = 0
    while 1: # Loop forever so the generator never terminates
                    
        if offset >= len(samples):
            offset = 0
        vals = np.zeros((batch_size, 1))
        imgs = np.zeros((batch_size, 160, 320, 3))
        index = 0
        while index < batch_size:
            batch_sample = samples[offset]
            offset += 1
            if offset >= len(samples):
                offset = 0
            if abs(float(batch_sample[1])) < .4 and random.random() < .5:
                continue
            filename = batch_sample[0]
            center_image = cv2.imread(filename)
            imgs[index] = center_image
            try:
                float(batch_sample[1])
            except:
                print(batch_sample[1])
            vals[index][0] = float(batch_sample[1])
            index += 1

        # trim image to only see section with road
        X_train = imgs
        y_train = vals
            
        yield X_train, y_train


### Define Hyperparameters

In [61]:
epochs = 50
batch_size = 64

train_data_size = len(train_data)
test_data_size = len(test_data)
validation_data_size = len(validation_data)

### Define Model and Train

In [62]:
train_generator = generator(train_data, batch_size=batch_size)
validation_generator = generator(validation_data, batch_size=batch_size)
test_generator = generator(test_data, batch_size=batch_size)

In [63]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Lambda, Flatten, Dropout
from keras.layers.convolutional import Conv2D, Cropping2D
from keras.layers.advanced_activations import ELU
import keras.regularizers as regularizers

model = Sequential()
model.add(Cropping2D(cropping=((55,30),(0,0)), input_shape=(160, 320, 3)))
model.add(Lambda(lambda x: x/127.5 - 1.0, input_shape=(75, 320, 3)))
model.add(Conv2D(16, (5,5), strides=(4, 4)))
model.add(ELU())
model.add(Conv2D(32, (5,5), strides=(2, 2)))
model.add(ELU())
model.add(Conv2D(64, (3,3), strides=(2, 2)))
model.add(ELU())
model.add(Conv2D(64, (3,3), strides=(2, 2)))
model.add(ELU())
model.add(Flatten())
model.add(Dropout(.2))
model.add(Dense(1024,kernel_regularizer=regularizers.l2(0.01)))
model.add(ELU())
model.add(Dense(512,kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(.5))
model.add(ELU())
model.add(Dense(64,kernel_regularizer=regularizers.l2(0.01)))
model.add(ELU())
model.add(Dense(1))


In [10]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Lambda, Flatten, Dropout
from keras.layers.convolutional import Conv2D, Cropping2D
from keras.layers.advanced_activations import ELU
from keras.optimizers import Adam

model = Sequential()
model.add(Cropping2D(cropping=((55,30),(0,0)), input_shape=(160, 320, 3)))
model.add(Lambda(lambda x: x/127.5 - 1.0, input_shape=(75, 320, 3)))
model.add(Conv2D(16, (8, 8), strides=(4, 4), padding="same"))
model.add(ELU())
model.add(Conv2D(32, (5, 5), strides=(2, 2), padding="same"))
model.add(ELU())
model.add(Conv2D(64, (5, 5), strides=(2, 2), padding="same"))
model.add(Flatten())
model.add(Dropout(.2))
model.add(ELU())
model.add(Dense(512))
model.add(Dropout(.5))
model.add(ELU())
model.add(Dense(1))

In [65]:
model.compile(optimizer=Adam(lr=0.0001), loss='mse', metrics=[ 'mse'])

In [ ]:
from keras.callbacks import ModelCheckpoint, Callback
checkpoint_callback = ModelCheckpoint('model{epoch:02d}.h5')
history = model.fit_generator(train_generator, steps_per_epoch=train_data_size/batch_size/3, \
                   validation_data=validation_generator, validation_steps=validation_data_size/batch_size, \
                   epochs=50, max_q_size=1, callbacks=[checkpoint_callback])

Epoch 1/50
62/61 [==============================] - 13s - loss: 13.3380 - mean_squared_error: 0.0696 - val_loss: 11.7421 - val_mean_squared_error: 0.0445
Epoch 2/50
62/61 [==============================] - 12s - loss: 10.3799 - mean_squared_error: 0.0485 - val_loss: 9.0529 - val_mean_squared_error: 0.0413
Epoch 3/50
62/61 [==============================] - 12s - loss: 7.9482 - mean_squared_error: 0.0420 - val_loss: 6.8896 - val_mean_squared_error: 0.0407
Epoch 4/50
62/61 [==============================] - 12s - loss: 6.0139 - mean_squared_error: 0.0376 - val_loss: 5.1860 - val_mean_squared_error: 0.0395
Epoch 5/50
62/61 [==============================] - 12s - loss: 4.5066 - mean_squared_error: 0.0372 - val_loss: 3.8668 - val_mean_squared_error: 0.0383
Epoch 6/50
62/61 [==============================] - 12s - loss: 3.3450 - mean_squared_error: 0.0350 - val_loss: 2.8584 - val_mean_squared_error: 0.0370
Epoch 7/50
62/61 [==============================] - 12s - loss: 2.4627 - mean_squared

### Evaluate

In [51]:
metrics = model.evaluate_generator(test_generator, steps=test_data_size/batch_size)

In [52]:
for i in zip(model.metrics_names, metrics):
    print(i)

('loss', 0.026486937329173087)
('acc', 0.56031249999999999)


In [53]:
model.save("model.h5")